In [1]:
from  get_data import *
from  placebo_simulations import *
# import  solver
from solver import *
from  synthdid import *

In [125]:
df = fetch_CaliforniaSmoking()
PRE_TEREM = [1970, 1988]
POST_TEREM = [1989, 2000]

TREATMENT = ["California"]

In [121]:
# def rmse_loss(W, X, y, intersept=True) -> float:
#     if type(y) == pd.core.frame.DataFrame:
#         y = y.mean(axis=1)
#     _X = X.copy()
#     if intersept:
#         _X["intersept"] = 1
#     return np.mean(np.sqrt((y - _X.dot(W)) ** 2))

In [122]:
# def rmse_loss_with_V(W, V, X, y) -> float:
#     if type(y) == pd.core.frame.DataFrame:
#         y = y.mean(axis=1)
#     _rss = (y - X.dot(W)) ** 2

#     _n = len(y)
#     _importance = np.zeros((_n, _n))

#     np.fill_diagonal(_importance, V)

#     return np.sum(_importance @ _rss)

In [123]:
# def _v_loss(V, X, y, Y_pre_t, Y_pre_c, return_loss=True):
#     Y_pre_t = Y_pre_t.copy()

#     n_features = Y_pre_c.shape[1]
#     _w = np.repeat(1 / n_features, n_features)

#     if type(Y_pre_t) == pd.core.frame.DataFrame:
#         Y_pre_t = Y_pre_t.mean(axis=1)

#     w_bnds = tuple((0, 1) for i in range(n_features))
#     _caled_w = fmin_slsqp(
#         partial(rmse_loss_with_V, V=V, X=X, y=y),
#         _w,
#         f_eqcons=lambda x: np.sum(x) - 1,
#         bounds=w_bnds,
#         disp=False,
#     )
#     if return_loss:
#         return rmse_loss(_caled_w, Y_pre_c, Y_pre_t, intersept=False)
#     else:
#         return _caled_w

In [124]:
# def estimate_v(additional_X, additional_y, Y_pre_t, Y_pre_c):
#     _len = len(additional_X)
#     _v = np.repeat(1 / _len, _len)

#     caled_v = fmin_slsqp(
#         partial(_v_loss, X=additional_X, y=additional_y, Y_pre_t = Y_pre_t, Y_pre_c = Y_pre_c),
#         _v,
#         f_eqcons=lambda x: np.sum(x) - 1,
#         bounds=tuple((0, 1) for i in range(_len)),
#         disp=False,
#     )
#     return caled_v

In [28]:
def est_omega_ADH(
        Y_pre_c, Y_pre_t, additional_X=pd.DataFrame(), additional_y=pd.DataFrame()
    ):
        """
        # SC
        estimating omega for synthetic control method (not for synthetic diff.-in-diff.)
        """
        Y_pre_t = Y_pre_t.copy()

        n_features = Y_pre_c.shape[1]
        nrow = Y_pre_c.shape[0]

        _w = np.repeat(1 / n_features, n_features)

        if type(Y_pre_t) == pd.core.frame.DataFrame:
            Y_pre_t = Y_pre_t.mean(axis=1)

        # Required to have non negative values
        w_bnds = tuple((0, 1) for i in range(n_features))

        if len(additional_X) == 0:
            caled_w = fmin_slsqp(
                partial(rmse_loss, X=Y_pre_c, y=Y_pre_t, intersept=False),
                _w,
                f_eqcons=lambda x: np.sum(x) - 1,
                bounds=w_bnds,
                disp=False,
            )

            return caled_w
        else:
            assert additional_X.shape[1] == Y_pre_c.shape[1]
            if type(additional_y) == pd.core.frame.DataFrame:
                additional_y = additional_y.mean(axis=1)

            # normalized
            temp_df = pd.concat([additional_X, additional_y], axis=1)
            ss = StandardScaler()
            ss_df = pd.DataFrame(
                ss.fit_transform(temp_df), columns=temp_df.columns, index=temp_df.index
            )

            ss_X = ss_df.iloc[:, :-1]
            ss_y = ss_df.iloc[:, -1]

            add_X = pd.concat([Y_pre_c, ss_X])
            add_y = pd.concat([Y_pre_t, ss_y])

            caled_v = estimate_v(additional_X=add_X, additional_y=add_y, Y_pre_t = Y_pre_t, Y_pre_c = Y_pre_c)

            return _v_loss(caled_v, X=add_X, y=add_y, Y_pre_t = Y_pre_t, Y_pre_c = Y_pre_c, return_loss=False)

In [3]:
sdid = gen_data(df, PRE_TEREM, POST_TEREM, TREATMENT)

In [126]:
sdid = synthdid_estimate(df, PRE_TEREM, POST_TEREM, TREATMENT)

In [65]:
def vcov(object, method = "placebo", replications = 200):
    Y_pre_c, Y_pre_t, Y_post_c, Y_post_t = sdid["Y_pre_c"], sdid["Y_pre_t"], sdid["Y_post_c"], sdid["Y_post_t"]
    pre_term, post_term = sdid["pre_term"], sdid["post_term"]

    if method == "placebo":

        assert sdid["n_treat"] < Y_pre_c.shape[1]
        control_names = Y_pre_c.columns

        result_tau_sdid = []

        for i in tqdm(range(replications)):
            # setup
            np.random.seed(seed=0 + i)
            placebo_t = np.random.choice(control_names, sdid["n_treat"], replace=False)
            placebo_c = [col for col in control_names if col not in placebo_t]
            pla_Y_pre_t = Y_pre_c[placebo_t]
            pla_Y_post_t = Y_post_c[placebo_t]
            pla_Y_pre_c = Y_pre_c[placebo_c]
            pla_Y_post_c = Y_post_c[placebo_c]

            pla_result = pd.DataFrame(
                {
                    "pla_actual_y": pd.concat([pla_Y_pre_t, pla_Y_post_t]).mean(
                        axis=1
                    )
                }
            )

            post_placebo_treat = pla_result.loc[
                post_term[0] :, "pla_actual_y"
            ].mean()

            # estimation
            ## sdid
            pla_zeta = est_zeta(sdid["n_treat"], sdid["n_post_term"], pla_Y_pre_c)

            pla_hat_omega = est_omega(l2_loss, pla_Y_pre_c, pla_Y_pre_t, pla_zeta)
            pla_hat_lambda = est_lambda(l2_loss, pla_Y_pre_c, pla_Y_post_c)

            # prediction
            ## sdid
            pla_hat_omega = pla_hat_omega[:-1]
            pla_Y_c = pd.concat([pla_Y_pre_c, pla_Y_post_c])
            n_features = pla_Y_pre_c.shape[1]
            start_w = np.repeat(1 / n_features, n_features)

            _intercept = (start_w - pla_hat_omega) @ pla_Y_pre_c.T @ pla_hat_lambda

            pla_result["sdid"] = pla_Y_c.dot(pla_hat_omega) + _intercept

            # cal tau
            ## sdid
            pre_sdid = pla_result["sdid"].head(len(pla_hat_lambda)) @ pla_hat_lambda
            post_sdid = pla_result.loc[post_term[0] :, "sdid"].mean()

            pre_treat = (pla_Y_pre_t.T @ pla_hat_lambda).values[0]
            sdid_counterfuctual_post_treat = pre_treat + (post_sdid - pre_sdid)

            result_tau_sdid.append(
                post_placebo_treat - sdid_counterfuctual_post_treat
            )
    return (np.var(result_tau_sdid))

In [62]:
np.var(result_tau_sdid)

79.58459898332936

In [97]:
np.random.choice(control_names, sdid["n_treat"], replace=False)

array(['South Carolina'], dtype=object)

In [115]:
algo = "jackknife"
# sdid["df"]

In [ ]:
Y_pre_c, Y_pre_t, Y_post_c, Y_post_t = sdid["Y_pre_c"], sdid["Y_pre_t"], sdid["Y_post_c"], sdid["Y_post_t"]
pre_term, post_term = sdid["pre_term"], sdid["post_term"]

if algo == "jackknife":

    assert sdid["n_treat"] < Y_pre_c.shape[1]
    control_names = Y_pre_c.columns

    result_tau_sdid = []
    
    if (setup$N0 == sdid["df"].shape[0] - 1 or (!is.null(weights) and np.sum(weights$omega != 0) == 1)):
        print("NA")

    for i in tqdm(range(replications)):
            # setup
        np.random.seed(seed=0 + i)
        placebo_t = np.random.choice(control_names, sdid["n_treat"], replace=False)
        placebo_c = [col for col in control_names if col not in placebo_t]
        pla_Y_pre_t = Y_pre_c[placebo_t]
        pla_Y_post_t = Y_post_c[placebo_t]
        pla_Y_pre_c = Y_pre_c[placebo_c]
        pla_Y_post_c = Y_post_c[placebo_c]

        pla_result = pd.DataFrame(
            {
                "pla_actual_y": pd.concat([pla_Y_pre_t, pla_Y_post_t]).mean(
                    axis=1
                )
            }
        )

        post_placebo_treat = pla_result.loc[
            post_term[0] :, "pla_actual_y"
        ].mean()

        # estimation
        ## sdid
        pla_zeta = est_zeta(sdid["n_treat"], sdid["n_post_term"], pla_Y_pre_c)

        pla_hat_omega = est_omega(l2_loss, pla_Y_pre_c, pla_Y_pre_t, pla_zeta)
        pla_hat_lambda = est_lambda(l2_loss, pla_Y_pre_c, pla_Y_post_c)
        ## sc
        # pla_hat_omega_ADH = est_omega_ADH(pla_Y_pre_c, pla_Y_pre_t)

        # prediction
        ## sdid
        pla_hat_omega = pla_hat_omega[:-1]
        pla_Y_c = pd.concat([pla_Y_pre_c, pla_Y_post_c])
        n_features = pla_Y_pre_c.shape[1]
        start_w = np.repeat(1 / n_features, n_features)

        _intercept = (start_w - pla_hat_omega) @ pla_Y_pre_c.T @ pla_hat_lambda

        pla_result["sdid"] = pla_Y_c.dot(pla_hat_omega) + _intercept

        # cal tau
        ## sdid
        pre_sdid = pla_result["sdid"].head(len(pla_hat_lambda)) @ pla_hat_lambda
        post_sdid = pla_result.loc[post_term[0] :, "sdid"].mean()

        pre_treat = (pla_Y_pre_t.T @ pla_hat_lambda).values[0]
        sdid_counterfuctual_post_treat = pre_treat + (post_sdid - pre_sdid)

        result_tau_sdid.append(
            post_placebo_treat - sdid_counterfuctual_post_treat
        )
np.var(result_tau_sdid)

In [111]:
pla_result

,pla_actual_y,sdid
1970,108.500000,120.397298
1971,108.400002,124.281081
1972,109.400002,129.713514
1973,110.599998,132.105405
1974,116.099998,135.170270
1975,120.500000,137.375676
1976,124.400002,141.716216
1977,125.500000,141.510810
1978,127.099998,140.835135
1979,124.199997,138.462163


In [ ]:
placebo_se = function(estimate, replications) {
    setup = attr(estimate, 'setup')
    opts = attr(estimate, 'opts')
    weights = attr(estimate, 'weights')
    N1 = nrow(setup$Y) - setup$N0
    if (setup$N0 <= N1) { stop('must have more controls than treated units to use the placebo se') }
    theta = function(ind) {
	N0 = length(ind)-N1
	weights.boot = weights
	weights.boot$omega = sum_normalize(weights$omega[ind[1:N0]])
        do.call(synthdid_estimate, c(list(Y=setup$Y[ind,], N0=N0,  T0=setup$T0,  X=setup$X[ind, ,], weights=weights.boot), opts))
    }
    sqrt((replications-1)/replications) * sd(replicate(replications, theta(sample(1:setup$N0))))
}

In [ ]:
theta = function(ind) {
N0 = length(ind)-N1
weights.boot = weights
weights.boot$omega = sum_normalize(weights$omega[ind[1:N0]])
    do.call(synthdid_estimate, c(list(Y=setup$Y[ind,], N0=N0,  T0=setup$T0,  X=setup$X[ind, ,], weights=weights.boot), opts))
}

In [ ]:
def theta(ind):
    N0 = len(ind) - N1
    weights_boot = weigths

In [6]:
len([1,2])

2